# Challenge for LUD22

In [1]:
import pandas as pd
import numpy as np
from rec import svd_decompose, svd_recommend_to_list
from rec import inter_matr_binary, split_interactions
from rec import recTopKPop
from rec import get_user_item_score

path = 'data_challenge/'

usr_path = path + 'MRSC_2022_demo.txt'
itm_path = path + 'MRSC_2022_tracks.txt'
inter_path = path + 'MRSC_2022_inter.txt'
target_usr_path = path + 'MRSC_2022_target_users.txt'

In [2]:

df_users = pd.read_csv(usr_path, sep='\t', header=None, names=['location', 'age', 'gender', 'date'])
df_items = pd.read_csv(itm_path, sep='\t', header=None, names=['artist', 'track'])

inter_matrix = inter_matr_binary(usr_path, itm_path, inter_path)


# Hypertuning Parameters

In [3]:
### Hypertuning
top_k = 15  #recommendations
n = 5  #amount of neighbors to consider

# Locating Users

In [4]:
users = np.loadtxt(target_usr_path, dtype=int)

In [5]:
from rec import recTopKPop



# Using K POP

In [6]:
for k, user in enumerate(users):

#    print(rec[k])

# Using RECTOPK

In [7]:
def recTopK(inter_matr: np.array,
            user: int,
            top_k: int,
            n: int) -> (np.array, np.array):
    '''
    inter_matr - np.array from the task 1
    user - user_id, integer
    top_k - expected length of the resulting list
    n - number of neighbors to consider

    returns - array of recommendations (sorted in the order of descending scores) & array of corresponding scores
    '''

    scores = np.zeros((inter_matr.shape[1],))

    for item in range(inter_matr.shape[1]):
        if inter_matr[user, item] == 0:
            score = get_user_item_score(inter_matr, user, item, n)
            scores[item] = score

    top_rec = (- scores).argsort()[:top_k]
    scores = scores[top_rec]

    return np.array(top_rec), np.array(scores)


In [32]:
# Adjustng out inter_matrix:

print(inter_matrix.shape)

# Assumption: Many data samples only are available once or twice -> Very unpopular and we can filter them out. Furthermore, this is supported by recTopKPop as the biggest value is 735. Bias in creating the matrix?

to_delete = []
for k, column in enumerate(inter_matrix.T):
    if column.sum() < 90:
        to_delete.append(k)

my_tmp_matrix_2 = np.delete(inter_matrix.T, to_delete, axis=0).T
new_matrix = np.delete(inter_matrix.T, to_delete, axis=0).T

print(my_tmp_matrix_2.shape)
to_delete = []
# delete users with few entries
my_tmp_matrix = np.delete(my_tmp_matrix_2, list(users), axis=0)

for k, row in enumerate(my_tmp_matrix):
    if row.sum() < 40:
        to_delete.append(k)
print(my_tmp_matrix.shape)
extra_matrix = np.delete(my_tmp_matrix, to_delete, axis=0)

print(extra_matrix.shape)


#my_2_matrix = my_matrix[list(users)]
#print(my_2_matrix.shape)

(12998, 9653)
(12998, 552)
(11699, 552)
(100, 552)


In [33]:
import multiprocessing

print(multiprocessing.cpu_count())


def calc_prediction(k, new_matrix, extra_matrix):
    if np.sum(new_matrix[k]) < 5:
        a = recTopKPop(inter_matrix, k, top_k)
    else:
        extra_matrix = np.vstack((new_matrix[k], extra_matrix))
        a, _ = recTopK(extra_matrix, 0, 15, 15)
        extra_matrix = np.delete(extra_matrix, 0, axis=0)
    return a

16


In [35]:
import multiprocessing as mp

rec = np.full((len(users), top_k), -1)

for k, user in enumerate(users):
    rec[k] = calc_prediction(k, new_matrix, extra_matrix)
    print(k,len(users))



0 1299
1 1299
2 1299
3 1299
4 1299
5 1299
6 1299
7 1299
8 1299
9 1299
10 1299
11 1299
12 1299
13 1299
14 1299
15 1299
16 1299
17 1299
18 1299
19 1299
20 1299
21 1299
22 1299
23 1299
24 1299
25 1299
26 1299
27 1299
28 1299
29 1299
30 1299
31 1299
32 1299
33 1299
34 1299
35 1299
36 1299
37 1299
38 1299
39 1299
40 1299
41 1299
42 1299
43 1299
44 1299
45 1299
46 1299
47 1299
48 1299
49 1299
50 1299
51 1299
52 1299
53 1299
54 1299
55 1299
56 1299
57 1299
58 1299
59 1299
60 1299
61 1299
62 1299
63 1299
64 1299
65 1299
66 1299
67 1299
68 1299
69 1299
70 1299
71 1299
72 1299
73 1299
74 1299
75 1299
76 1299
77 1299
78 1299
79 1299
80 1299
81 1299
82 1299
83 1299
84 1299
85 1299
86 1299
87 1299
88 1299
89 1299
90 1299
91 1299
92 1299
93 1299
94 1299
95 1299
96 1299
97 1299
98 1299
99 1299
100 1299
101 1299
102 1299
103 1299
104 1299
105 1299
106 1299
107 1299
108 1299
109 1299
110 1299
111 1299
112 1299
113 1299
114 1299
115 1299
116 1299
117 1299
118 1299
119 1299
120 1299
121 1299
122 1299
123

In [39]:
with open('rec_k12006268_Viktor_loreth.tsv', 'w') as f:
    for k, row in enumerate(rec):
        f.write(f'{users[k]}\t')
        for ele in row:
            f.write(f'{int(ele)}')
            if ele != row[-1]:
                f.write(f',')

        f.write('\n')



In [17]:
print(rec)

[[1363 2256  189 ... 1913  237  247]
 [1078  857  919 ...  431   97  344]
 [ 403  627  661 ... 2098 2562 2554]
 ...
 [ 889  496  653 ...  710 1305 1639]
 [   3  187 1170 ... 1262  154 1028]
 [  28 1255 2689 ...  895 2076 2558]]
